In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.pipeline import Pipeline

In [2]:
# Load the data
csv_path = 'predictive_maintenance_demo_data.csv'
df = pd.read_csv(csv_path)

In [3]:
df

,Asset ID,Asset Type,Purchase Date,Usage Hours,Maintenance Count,Last Maintenance Date,Environmental Temperature,Humidity,Age Days,Failure
0,1,Building,16-04-2012,8434,15,17-08-2019,24.870932,62.525939,4484,1
1,2,Vehicle,10-07-2016,2590,1,02-12-2018,29.045175,20.501821,2938,0
2,3,Building,05-06-2018,1063,5,23-08-2018,34.856337,33.060044,2243,0
3,4,Building,27-01-2015,4324,11,31-12-2010,17.629782,59.701057,3468,0
4,5,Vehicle,30-12-2014,9111,4,12-08-2011,20.494631,49.039362,3496,1
...,...,...,...,...,...,...,...,...,...,...
995,996,Equipment,26-05-2012,9153,16,02-11-2019,16.374274,57.805588,4444,1
996,997,Equipment,09-01-2016,2989,18,08-06-2013,22.391484,69.719174,3121,0
997,998,Building,18-06-2016,9218,6,12-10-2010,24.284770,67.008174,2960,1
998,999,Building,08-08-2019,6228,8,11-07-2017,29.454755,36.624155,1814,0


In [4]:
# Preprocess the data
df['Purchase Date'] = pd.to_datetime(df['Purchase Date'])
df['Last Maintenance Date'] = pd.to_datetime(df['Last Maintenance Date'])

C:\Users\DELL\AppData\Local\Temp\ipykernel_12388\553544601.py:2: UserWarning: Parsing '16-04-2012' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['Purchase Date'] = pd.to_datetime(df['Purchase Date'])
C:\Users\DELL\AppData\Local\Temp\ipykernel_12388\553544601.py:2: UserWarning: Parsing '27-01-2015' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['Purchase Date'] = pd.to_datetime(df['Purchase Date'])
C:\Users\DELL\AppData\Local\Temp\ipykernel_12388\553544601.py:2: UserWarning: Parsing '30-12-2014' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['Purchase Date'] = pd.to_datetime(df['Purchase Date'])
C:\Users\DELL\AppData\Local\Temp\ipykernel_12388\553544601.py:2: UserWarning: Parsing '23-08-2011' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['Purchase Date'] = pd.t

In [5]:
# Feature engineering
df['Asset Age Days'] = (pd.to_datetime('today') - df['Purchase Date']).dt.days
df['Days Since Last Maintenance'] = (pd.to_datetime('today') - df['Last Maintenance Date']).dt.days

In [6]:
# Select features and target variable
features = ['Usage Hours', 'Maintenance Count', 'Environmental Temperature', 'Humidity', 'Asset Age Days', 'Days Since Last Maintenance']
target = 'Failure'

In [7]:
X = df[features]
y = df[target]

In [8]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
# Create a pipeline with a scaler and classifier
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

In [10]:
# Train the model
pipeline.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('classifier', RandomForestClassifier(random_state=42))])

In [11]:
# Make predictions
y_pred = pipeline.predict(X_test)
y_pred_proba = pipeline.predict_proba(X_test)[:, 1]

In [12]:
# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 1.0
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       158
           1       1.00      1.00      1.00        42

    accuracy                           1.00       200
   macro avg       1.00      1.00      1.00       200
weighted avg       1.00      1.00      1.00       200



In [13]:
# Example output for a single prediction
example_input = np.array([[5000, 5, 25, 50, 2000, 180]])  # Example feature values
example_pred = pipeline.predict(example_input)
example_pred_proba = pipeline.predict_proba(example_input)[:, 1]

C:\Users\DELL\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\DELL\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [14]:
print("Prediction (0=No Maintenance, 1=Maintenance Needed):", example_pred)
print("Prediction Probability:", example_pred_proba)

Prediction (0=No Maintenance, 1=Maintenance Needed): [0]
Prediction Probability: [0.05]


In [15]:
import joblib

In [16]:
# Save the model pipeline to a .pkl file
model_filename = 'predictive_maintenance_model.pkl'
joblib.dump(pipeline, model_filename)

['predictive_maintenance_model.pkl']